In [680]:
!python --version

Python 3.12.4


In [681]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from dateutil import parser
from datetime import datetime


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import Birch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering, AgglomerativeClustering, KMeans
from kmodes.kprototypes import KPrototypes

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import cross_val_score ,StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, RobustScaler, MinMaxScaler, StandardScaler

le = LabelEncoder()
skf10 = StratifiedKFold(n_splits=10, shuffle=True,random_state=24)
skf5 = StratifiedKFold(n_splits=5, shuffle=True,random_state=24)

from lofo import LOFOImportance, Dataset, plot_importance
from sklearn.feature_selection import RFECV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import optuna
from scipy.stats import mode
from itertools import combinations
from tqdm.notebook import tqdm
tqdm.pandas()

import unicodedata


# 1. Veriye İlk Bakış

In [682]:
test       = pd.read_csv('data\\test_x.csv').drop(['id'],axis=1)
train      = pd.read_csv('data\\train.csv').drop(['id'],axis=1)
train_copy = pd.read_csv('data\\train.csv').drop(['id'],axis=1)

target_col = 'Degerlendirme Puani'
categoric_cols = [col for col in train.columns if train[col].dtype == 'O']
numeric_cols = [col for col in train.columns if train[col].dtype != 'O']
numeric_cols.remove(target_col)

In [683]:
train

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite"
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,3,3.50-3,Hayır,Ankara Ataturk Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.50-3,Hayır,NaN,NaN,Üniversite,Hayır,0,Üniversite,Emekli,0,1,Evet,Bilkent IEEE Bilgisayar Toplulugu 2013-2014 Ba...,Evet,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,3,3.00-2.50,Hayır,Betül Can Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.00 - 4.00,Hayır,NaN,NaN,Üniversite,Evet,Kamu,Yüksek Lisans / Doktora,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,1,NaN,Hayır,Batem,NaN,Ankara,Meslek lisesi,Elektirk,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,İlkokul Mezunu,Evet,Özel Sektör,6,Evet,Tema,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,3,3.00-2.50,Hayır,Cumhuriyet Fen Lisesi,NaN,Diyarbakır,Fen lisesi,Sayısal,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,10,Evet,Güzel Konuşma ve yazma kulübü,Hayır,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,NaN,3.00-2.50,Hayır,Samanyolu Lisesi,NaN,Ankara,Özel lisesi,TM,NaN,3.00 - 4.00,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,2,0 - 1.79,NaN,BAĞCILAR EDİP İPLİK MESLEKİ VE TEKNİK ANADOLU ...,NaN,İstanbul,Devlet,Eşit Ağırlık,NaN,50 - 74,Hayır,-,-,İlkokul,Hayır,-,Ortaokul,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,1,Hazırlığım,NaN,Ahmet Vefik paşa Anadolu lisesi,NaN,Bursa,Devlet,Eşit Ağırlık,NaN,75 - 100,Hayır,-,-,Lise,Hayır,-,Lise,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,4,0 - 1.79,NaN,Açık lise,NaN,Mersin,Devlet,Sayısal,NaN,75 - 100,Evet,Kyk,500₺ - 999₺,Lise,Hayır,-,Lise,Hayır,-,3.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,3,0 - 1.79,NaN,Samsun Terme Karadeniz mesleki teknik Anadolu ...

In [684]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 43 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu

In [685]:
train.isnull().sum().sort_values(ascending=False)/65125*100

Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite        99.356622
Lise Bolum Diger                                             97.656814
Lise Adi Diger                                               92.732438
Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?    77.583109
Hangi STK'nin Uyesisiniz?                                    76.035317
Uye Oldugunuz Kulubun Ismi                                   74.765451
Baska Kurumdan Aldigi Burs Miktari                           67.811132
Burs Aldigi Baska Kurum                                      67.527063
Burslu ise Burs Yuzdesi                                      65.543186
Daha Once Baska Bir Universiteden Mezun Olmus                57.343570
Ingilizce Seviyeniz?                                         56.096737
Anne Sektor                                                  53.031862
Spor Dalindaki Rolunuz Nedir?                                49.558541
Stk Projesine Katildiniz Mi?                                 43.467179
Baba S

In [686]:
train[categoric_cols].describe().T

,count,unique,top,freq
Cinsiyet,64956,4,Kadın,32077
Dogum Tarihi,64948,13498,1/1/70 2:00,1655
Dogum Yeri,64334,2230,İstanbul,8682
Ikametgah Sehri,63088,319,İstanbul,16625
Universite Adi,64993,527,İSTANBUL ÜNİVERSİTESİ,2083
Universite Turu,64870,4,Devlet,37885
Burs Aliyor mu?,65125,5,Hayır,51978
Bölüm,64894,7748,Endüstri Mühendisliği,2434
Universite Kacinci Sinif,64751,12,2,21773
Universite Not Ortalamasi,62372,20,3.00-2.50,10758


In [687]:
train[numeric_cols+[target_col]].describe().T

,count,mean,std,min,25%,50%,75%,max
Basvuru Yili,65125.0,2018.277298,2.669979,2014.0,2016.0,2019.0,2021.0,2022.0
Burslu ise Burs Yuzdesi,22440.0,34.226181,41.632412,0.0,0.0,0.0,75.0,100.0
Degerlendirme Puani,65124.0,32.086466,18.139239,0.0,18.0,29.0,44.0,102.0


# 2. Veriyi Temizleme

## 2.1 Basvuru Yili

In [688]:
train[["Basvuru Yili"]].isna().sum()

Basvuru Yili    0
dtype: int64

In [689]:
train[["Basvuru Yili"]] = train[["Basvuru Yili"]].astype(str)

train["Basvuru Yili"].dtype

#Label Encoder kullanırız sonra belki bence dataya girmemesi gerekiyor

dtype('O')

## 2.2 Cinsiyet

In [690]:
train[["Cinsiyet"]].isna().sum()

Cinsiyet    169
dtype: int64

In [691]:
train['Cinsiyet'].value_counts()

Cinsiyet
Kadın                    32077
Erkek                    22752
ERKEK                    10012
Belirtmek istemiyorum      115
Name: count, dtype: int64

In [692]:
train['Cinsiyet'] = train['Cinsiyet'].str.capitalize()
train['Cinsiyet'].fillna('Belirtmek istemiyorum', inplace=True)
train['Cinsiyet'].value_counts()


Cinsiyet
Erkek                    32764
Kadın                    32077
Belirtmek istemiyorum      284
Name: count, dtype: int64

In [693]:
# One-hot encoding
#train = pd.get_dummies(train, columns=['Cinsiyet'], drop_first=True)
#train

#label encodin

## 2.3 Dogum Tarihi - Dogum Yili 

In [694]:
# Mapping of Turkish month names to English
turkish_to_english_months = {
    "Ocak": "January",
    "Şubat": "February",
    "Subat": "February",
    "Mart": "March",
    "Nisan": "April",
    "Mayıs": "May",
    "Mayis": "May",
    "Haziran": "June",
    "Temmuz": "July",
    "Ağustos": "August",
    "Agustos": "August",
    "Eylül": "September",
    "Eylul": "September",
    "Ekim": "October",
    "Kasım": "November",
    "Kasim": "November",
    "Aralık": "December",
    "Aralik": "December"
}

def normalize_turkish_months(date_str):
    for turkish_month, english_month in turkish_to_english_months.items():
        date_str = re.sub(r'\b' + turkish_month + r'\b', english_month, date_str)
    return date_str

def extract_year(date_str):
    if not isinstance(date_str, str):
        return None  # Return None if the input is not a string
    try:
        # Normalize the date string for Turkish month names
        date_str = normalize_turkish_months(date_str)
        
        # Parse the date string using dateutil.parser
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=False)
        
        # Extract the year from the parsed date
        return parsed_date.year
    except (ValueError, TypeError):
        # If parsing fails, try extracting the year directly from the string
        year_match = re.search(r'(\d{4})$', date_str)
        if year_match:
            return int(year_match.group(1))
        return None
    


train['Dogum Yili'] = train['Dogum Tarihi'].apply(extract_year)
train['Dogum Yili'].isnull().sum()


192

In [695]:
train['Dogum Yili'].value_counts()

Dogum Yili
2000.0    7189
1999.0    6690
1998.0    6623
1995.0    6301
1996.0    6158
1997.0    6122
2001.0    5675
1994.0    5398
1993.0    4165
2002.0    3212
1992.0    2585
2070.0    1661
1991.0    1397
2003.0     737
1990.0     601
1989.0      81
1970.0      55
1988.0      49
1987.0      32
2016.0      24
2004.0      23
1986.0      21
2017.0      11
2005.0       9
2020.0       9
199.0        8
1984.0       7
1985.0       6
1197.0       6
200.0        5
1976.0       5
1983.0       5
1980.0       4
1196.0       4
1977.0       3
197.0        3
1979.0       2
996.0        2
9691.0       2
9951.0       2
998.0        2
1199.0       2
2011.0       2
993.0        2
1195.0       2
2015.0       1
7199.0       1
328.0        1
2006.0       1
1204.0       1
808.0        1
2009.0       1
198.0        1
989.0        1
9941.0       1
6123.0       1
1929.0       1
1198.0       1
9590.0       1
1155.0       1
5778.0       1
1194.0       1
2996.0       1
1896.0       1
9916.0       1
1888.0       1

In [696]:
train['Dogum Yili'] = train['Dogum Yili'].apply(lambda x: np.nan if (x > 2005) | (x < 1980) else x)
train['Dogum Yili'].isnull().sum()

2033

In [697]:
train['Dogum Yili'].value_counts()

Dogum Yili
2000.0    7189
1999.0    6690
1998.0    6623
1995.0    6301
1996.0    6158
1997.0    6122
2001.0    5675
1994.0    5398
1993.0    4165
2002.0    3212
1992.0    2585
1991.0    1397
2003.0     737
1990.0     601
1989.0      81
1988.0      49
1987.0      32
2004.0      23
1986.0      21
2005.0       9
1984.0       7
1985.0       6
1983.0       5
1980.0       4
1982.0       1
1981.0       1
Name: count, dtype: int64

## 2.4 Basvuru Yasi

In [698]:
train['Basvuru Yasi'] = train['Basvuru Yili'].astype(float) - train['Dogum Yili']
train['Basvuru Yasi'].value_counts()

Basvuru Yasi
21.0    15764
20.0    15046
22.0    12487
19.0     8085
23.0     7134
24.0     3073
25.0      781
18.0      394
26.0      117
17.0       58
27.0       56
28.0       35
30.0       20
29.0       13
31.0        8
34.0        7
32.0        3
33.0        3
16.0        2
35.0        2
14.0        1
11.0        1
36.0        1
40.0        1
Name: count, dtype: int64

In [699]:
train['Basvuru Yasi'].isnull().sum()

2033

## 2.5 Dogum Yeri, Ikametgah Sehri, Lise Sehir

In [700]:
print(f"Dogum Yeri none veri sayisi : {train["Dogum Yeri"].isna().sum()}")

Dogum Yeri none veri sayisi : 791


In [701]:
train["Dogum Yeri"].value_counts()

Dogum Yeri
İstanbul             8682
Ankara               4045
İzmir                2170
Istanbul             1894
Adana                1810
                     ... 
körfez/kocaeli          1
GEDIZ                   1
İstanbul Bakırköy       1
İstanbul-Şişli          1
Ankara keçiören         1
Name: count, Length: 2230, dtype: int64

In [702]:
def normalize_turkish_with_unicode(text):

    if text =="nan" or text=='np.nan' or text == 'none':
        return np.nan  # Keep np.nan as it is
    # Normalize the text to decompose combined characters (NFKD form)
    normalized_text = unicodedata.normalize('NFKD', text)
    
    # Remove diacritical marks (characters with combining marks)
    ascii_text = ''.join(c for c in normalized_text if not unicodedata.combining(c))
    
    # Additional replacements for Turkish-specific characters
    turkish_to_ascii = {
        'ç': 'c', 'Ç': 'C',
        'ğ': 'g', 'Ğ': 'G',
        'ı': 'i', 'İ': 'i',
        'ö': 'o', 'Ö': 'O',
        'ş': 's', 'Ş': 'S',
        'ü': 'u', 'Ü': 'U'
    }
    
    # Replace Turkish characters with ASCII equivalents
    ascii_text = ''.join(turkish_to_ascii.get(c, c) for c in ascii_text)
    
    return ascii_text.lower()

def temizle_sehir(sehir):
    if pd.isna(sehir):
        return None
    if isinstance(sehir, str):
        sehir = sehir.lower()
        sehir = sehir.strip()

        for il in sehirler:
            if il.lower() in sehir:
                return il.capitalize()
    return None

sehirler = [
    "Adana", "Adıyaman", "Afyonkarahisar", "Ağrı", "Aksaray", "Amasya", "Ankara", "Antalya",
    "Ardahan", "Artvin", "Aydın", "Balıkesir", "Bartın", "Batman", "Bayburt", "Bilecik", "Bingöl",
    "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale", "Çankırı", "Çorum", "Denizli", "Diyarbakır",
    "Düzce", "Edirne", "Elazığ", "Erzincan", "Erzurum", "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane",
    "Hakkari", "Hatay", "Iğdır", "Isparta", "İstanbul", "İzmir", "Kahramanmaraş", "Karabük", "Karaman",
    "Kars", "Kastamonu", "Kayseri", "Kilis", "Kırıkkale", "Kırklareli", "Kırşehir", "Kocaeli", "Konya",
    "Kütahya", "Malatya", "Manisa", "Mardin", "Mersin", "Muğla", "Muş", "Nevşehir", "Niğde", "Ordu",
    "Osmaniye", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", "Şanlıurfa", "Şırnak", "Tekirdağ",
    "Tokat", "Trabzon", "Tunceli", "Uşak", "Van", "Yalova", "Yozgat", "Zonguldak"
]

sehirler = list(map(normalize_turkish_with_unicode,sehirler))

sehir_col_list = ["Dogum Yeri", "Ikametgah Sehri", "Lise Sehir"]

for col in sehir_col_list:

    print(f"{col} none veri sayisi önceki : {train[col].isna().sum()}")
    train[col] = train[col].astype(str).apply(normalize_turkish_with_unicode)

    train[col] = train[col].apply(temizle_sehir)

    print(f"{col} none veri sayisi : {train[col].isna().sum()}")

    print(train[col].value_counts().head())

Dogum Yeri none veri sayisi önceki : 791
Dogum Yeri none veri sayisi : 3267
Dogum Yeri
Istanbul    11072
Ankara       4275
Izmir        2894
Bursa        1931
Adana        1914
Name: count, dtype: int64
Ikametgah Sehri none veri sayisi önceki : 2037
Ikametgah Sehri none veri sayisi : 2423
Ikametgah Sehri
Istanbul    20228
Ankara       6602
Izmir        3893
Bursa        1794
Kocaeli      1472
Name: count, dtype: int64
Lise Sehir none veri sayisi önceki : 1054
Lise Sehir none veri sayisi : 1845
Lise Sehir
Istanbul    13342
Ankara       5165
Izmir        3470
Bursa        2360
Adana        1917
Name: count, dtype: int64


## 2.7 Universite Adi

In [703]:
print(f"Universite Adi none veri sayisi : {train["Universite Adi"].isna().sum()}")

Universite Adi none veri sayisi : 132


In [704]:
train['Universite Adi'] = train['Universite Adi'].astype(str).apply(normalize_turkish_with_unicode)


print(f"Universite Adi none veri sayisi : {train["Universite Adi"].isna().sum()}")

print(f"Universite Adi nunique veri sayisi : {train['Universite Adi'].value_counts().shape[0]}")

print(train["Universite Adi"].value_counts().head())

Universite Adi none veri sayisi : 132
Universite Adi nunique veri sayisi : 291
Universite Adi
istanbul universitesi            2572
marmara universitesi             2084
istanbul teknik universitesi     1954
yildiz teknik universitesi       1916
orta dogu teknik universitesi    1658
Name: count, dtype: int64


## 2.8 Universite Turu

Baktım uni türü nan olanların cogunun unisinin adı var ama data az çok önemli değil

In [705]:
print(f"Universite Turu none veri sayisi : {train["Universite Turu"].isna().sum()}")
train['Universite Turu'].value_counts()

Universite Turu none veri sayisi : 255


Universite Turu
Devlet    37885
DEVLET    15377
Özel       8336
ÖZEL       3272
Name: count, dtype: int64

In [706]:
train['Universite Turu'].fillna("Devlet", inplace=True)

train['Universite Turu'] = train['Universite Turu'].str.capitalize()

print(f"Universite Turu none veri sayisi : {train["Universite Turu"].isna().sum()}")
train['Universite Turu'].value_counts() 

Universite Turu none veri sayisi : 0


Universite Turu
Devlet    53517
Özel      11608
Name: count, dtype: int64

## 2.9 Baska Bir Kurumdan Burs Aliyor mu? , Burslu ise Burs Yuzdesi

Burs yüzdesini çok random çok nan var bu direk drop.

In [707]:
print(f"Baska Bir Kurumdan Burs Aliyor mu? none veri sayisi : {train['Baska Bir Kurumdan Burs Aliyor mu?'].isna().sum()}")
train['Baska Bir Kurumdan Burs Aliyor mu?'].value_counts().head(10)

Baska Bir Kurumdan Burs Aliyor mu? none veri sayisi : 231


Baska Bir Kurumdan Burs Aliyor mu?
Hayır    48513
Evet     16381
Name: count, dtype: int64

In [708]:
print(f"urslu ise Burs Yuzdesi none veri sayisi : {train["Burslu ise Burs Yuzdesi"].isna().sum()}")
train['Burslu ise Burs Yuzdesi'].value_counts().head(10)

urslu ise Burs Yuzdesi none veri sayisi : 42685


Burslu ise Burs Yuzdesi
0.0      12172
100.0     5049
50.0      3149
75.0      1002
25.0       785
10.0        50
60.0        27
1.0         26
65.0        17
80.0        15
Name: count, dtype: int64

In [709]:
train.groupby(['Baska Bir Kurumdan Burs Aliyor mu?']).agg(ortalama_burs=('Burslu ise Burs Yuzdesi', 'mean')).reset_index()

,Baska Bir Kurumdan Burs Aliyor mu?,ortalama_burs
0,Evet,60.538128
1,Hayır,28.563572


In [710]:
train.loc[train['Baska Bir Kurumdan Burs Aliyor mu?']=="Hayır"]['Burslu ise Burs Yuzdesi'].value_counts().head(10)



Burslu ise Burs Yuzdesi
0.0      11074
100.0     3125
50.0      2615
75.0       792
25.0       675
60.0        26
10.0        25
1.0         15
80.0        13
5.0         13
Name: count, dtype: int64

In [711]:
train.loc[train['Baska Bir Kurumdan Burs Aliyor mu?']=="Hayır"]['Burslu ise Burs Yuzdesi'].isna().sum()

30030

In [712]:
train.loc[train['Baska Bir Kurumdan Burs Aliyor mu?']=="Evet"]['Burslu ise Burs Yuzdesi'].value_counts().head(10)

Burslu ise Burs Yuzdesi
100.0    1906
0.0      1098
50.0      519
75.0      208
25.0      109
10.0       25
1.0        11
65.0        5
45.0        4
55.0        3
Name: count, dtype: int64

In [713]:
train.loc[train['Baska Bir Kurumdan Burs Aliyor mu?']=="Evet"]['Burslu ise Burs Yuzdesi'].isna().sum()

12460

In [714]:
train['Baska Bir Kurumdan Burs Aliyor mu?'].fillna("Hayır", inplace=True)
train['Baska Bir Kurumdan Burs Aliyor mu?'].value_counts().head(10)

Baska Bir Kurumdan Burs Aliyor mu?
Hayır    48744
Evet     16381
Name: count, dtype: int64

## 2.10 Bölüm

In [715]:
print(f"Bölüm none veri sayisi : {train["Bölüm"].isna().sum()}")

Bölüm none veri sayisi : 231


In [716]:
train['Bölüm'] = train['Bölüm'].astype(str).apply(normalize_turkish_with_unicode)


print(f"Bölüm none veri sayisi : {train["Bölüm"].isna().sum()}")

print(f"Bölüm nunique veri sayisi : {train['Bölüm'].value_counts().shape[0]}")

print(train["Bölüm"].value_counts().head(25))

Bölüm none veri sayisi : 231
Bölüm nunique veri sayisi : 5244
Bölüm
isletme                                     2913
endustri muhendisligi                       2835
bilgisayar muhendisligi                     2245
hukuk                                       2007
makine muhendisligi                         1552
iktisat                                     1529
tip                                         1334
insaat muhendisligi                         1206
uluslararasi iliskiler                       974
elektrik-elektronik muhendisligi             957
diger                                        957
elektrik elektronik muhendisligi             830
mimarlik                                     826
hemsirelik                                   804
psikoloji                                    758
siyaset bilimi ve kamu yonetimi              715
ilahiyat                                     601
maliye                                       596
sosyoloji                                    586
e

In [717]:
def temizle_bolum(bolum):
    if pd.isna(bolum):
        return None
    if isinstance(bolum, str):
        bolum = bolum.lower()
        bolum = bolum.strip()

        for bolum_i in bolumler:
            if bolum_i.lower() in bolum:
                return bolum_i.lower()
    return bolum

bolumler = [
    "makine", "endustri", "elektronik","insaat","mekatronik","metalurji","cevre","petrol","maden","gida","ucak",
    "Uzay","robotik","otomotiv","tekstil","bilgisayar muhendisligi","yazilim"
    ]

train['Bölüm'] = train['Bölüm'].str.replace('-', '').str.strip()
train["Bölüm"] = train["Bölüm"].apply(temizle_bolum)
print(train["Bölüm"].value_counts().head(25).tail(25))

Bölüm
endustri                           4368
elektronik                         3164
isletme                            2913
bilgisayar muhendisligi            2773
makine                             2165
hukuk                              2007
insaat                             1605
iktisat                            1529
tip                                1334
uluslararasi iliskiler              974
diger                               957
mimarlik                            826
hemsirelik                          804
psikoloji                           758
siyaset bilimi ve kamu yonetimi     715
ilahiyat                            601
maliye                              596
sosyoloji                           586
mekatronik                          578
eczacilik                           567
dis hekimligi                       561
gida                                503
cevre                               490
metalurji                           472
yonetim bilisim sistemleri        

## 2.11 Universite Kacinci Sinif

In [718]:
print(f"Universite Kacinci Sinif none veri sayisi : {train["Universite Kacinci Sinif"].isna().sum()}")

Universite Kacinci Sinif none veri sayisi : 374


In [719]:
train["Universite Kacinci Sinif"] = train["Universite Kacinci Sinif"].astype(str).apply(normalize_turkish_with_unicode)

train["Universite Kacinci Sinif"].value_counts()

Universite Kacinci Sinif
2                21773
3                16956
4                13113
1                10260
hazirlik          1308
5                  961
mezun              178
6                  177
yuksek lisans       15
tez                  7
0                    3
Name: count, dtype: int64

In [720]:
uni_sinif_mapping = {'1': 1, '2': 2, '3': 3,'4': 4, '5': 5,'6':5,'yuksek lisans':7,'tez':7,'mezun':6,"hazirlik":0, "0":0}

In [721]:
train['Universite Kacinci Sinif'] = train['Universite Kacinci Sinif'].map(uni_sinif_mapping)

In [722]:
train['Universite Kacinci Sinif'].value_counts()

Universite Kacinci Sinif
2.0    21773
3.0    16956
4.0    13113
1.0    10260
0.0     1311
5.0     1138
6.0      178
7.0       22
Name: count, dtype: int64

## 2.12 Universite Not Ortalamasi

In [723]:
print(f"Universite Not Ortalamasi none veri sayisi : {train["Universite Not Ortalamasi"].isna().sum()}")

train["Universite Not Ortalamasi"].value_counts()

Universite Not Ortalamasi none veri sayisi : 2753


Universite Not Ortalamasi
3.00-2.50              10758
2.50 ve altı           10554
3.00 - 3.50             7913
3.50-3                  6179
2.50 - 3.00             5782
3.50 - 4.00             4462
2.00 - 2.50             2519
2.50 - 2.99             2452
ORTALAMA BULUNMUYOR     2399
3.00 - 3.49             2382
2.50 -3.00              1734
3.00 - 4.00             1411
1.80 - 2.49             1166
Hazırlığım               668
Not ortalaması yok       586
4-3.5                    481
Ortalama bulunmuyor      413
1.00 - 2.50              327
0 - 1.79                 185
4.0-3.5                    1
Name: count, dtype: int64

In [724]:
uni_not_mapping = {'ORTALAMA BULUNMUYOR': 0, 'Not ortalaması yok': 0, 'Hazırlığım': 0,'Ortalama bulunmuyor': 0,
                      "0 - 1.79": 1,
                      "1.00 - 2.50":2,"1.80 - 2.49":2,"2.50 ve altı":2,"2.00 - 2.50":2,
                      "3.00-2.50":3,"2.50 -3.00":3,"2.50 - 2.99":3,"2.50 - 3.00":3,
                      "3.00 - 3.50":4,"3.00 - 4.00":4,"3.00 - 3.49":4,"3.50-3":4,
                      "4-3.5":5,"4-3.5":5,"3.50 - 4.00":5,"4.0-3.5":5}

train['Universite Not Ortalamasi'] = train['Universite Not Ortalamasi'].map(uni_not_mapping)

In [725]:
print(f"Universite Not Ortalamasi none veri sayisi : {train["Universite Not Ortalamasi"].isna().sum()}")
train["Universite Not Ortalamasi"].value_counts()

Universite Not Ortalamasi none veri sayisi : 2753


Universite Not Ortalamasi
3.0    20726
4.0    17885
2.0    14566
5.0     4944
0.0     4066
1.0      185
Name: count, dtype: int64

## 2.13 Daha Once Baska Bir Universiteden Mezun Olmus

In [726]:
print(f"Daha Once Baska Bir Universiteden Mezun Olmus none veri sayisi : {train["Daha Once Baska Bir Universiteden Mezun Olmus"].isna().sum()}")
train["Daha Once Baska Bir Universiteden Mezun Olmus"].value_counts()

Daha Once Baska Bir Universiteden Mezun Olmus none veri sayisi : 37345


Daha Once Baska Bir Universiteden Mezun Olmus
Hayır    26624
Evet      1156
Name: count, dtype: int64

In [727]:
train["Daha Once Baska Bir Universiteden Mezun Olmus"].fillna('Hayır', inplace=True)
train["Daha Once Baska Bir Universiteden Mezun Olmus"].value_counts()

Daha Once Baska Bir Universiteden Mezun Olmus
Hayır    63969
Evet      1156
Name: count, dtype: int64

## 2.14 Lise Adi Diger, Lise Turu

In [728]:
print(f"Lise Adi none veri sayisi : {train["Lise Adi"].isna().sum()}")
train["Lise Adi"].value_counts()

Lise Adi none veri sayisi : 384


Lise Adi
------                                                                             4251
Atatürk Lisesi                                                                      199
Açık öğretim lisesi                                                                 161
Açık Öğretim Lisesi                                                                 151
Cumhuriyet Anadolu Lisesi                                                           146
                                                                                   ... 
Nişantaşı Işık Lisesi                                                                 1
Tarsus atatürk lisesi                                                                 1
Hüsniye Özdilek Ticaret ve İletişim Anadolu Lisesi (Radyo ve Televizyon Bölümü)       1
Ses okulları                                                                          1
Samsun Terme Karadeniz mesleki teknik Anadolu lisesi                                  1
Name: count, Length: 31

In [729]:
print(f"Lise Adi Diger none veri sayisi : {train["Lise Adi Diger"].isna().sum()}")
train["Lise Adi Diger"].value_counts()

Lise Adi Diger none veri sayisi : 60392


Lise Adi Diger
Atatürk Anadolu Lisesi         8
Beşiktaş Anadolu Lisesi        7
Açıköğretim lisesi             6
Uşak Fen Lisesi                6
Kocaeli Anadolu Lisesi         6
                              ..
Seyyid Burhaneddin ATL         1
Süleyman Çelebi                1
Bitls Ticaret meslek lisesi    1
Van Fen Lisesi                 1
büyük çiğli anadolu lisesi     1
Name: count, Length: 4010, dtype: int64

In [730]:
print(train[train["Lise Adi"] == "------"]["Lise Adi Diger"].head())
print(f"Lise Adi Diger none veri sayisi : {train[train["Lise Adi"] == "------"]["Lise Adi Diger"].isna().sum()}")

603                   İstanbul Alman Lisesi
605           Mugla Anadolu öğretmen Lisesi
607                     imkb anadalo lisesi
609    Orta Doğu Teknik Üniversitesi Koleji
616                   Diltaş Anadolu Lisesi
Name: Lise Adi Diger, dtype: object
Lise Adi Diger none veri sayisi : 275


In [731]:
train.loc[train["Lise Adi"] == '------',"Lise Adi"] = train["Lise Adi Diger"]
print(f"Lise Adi none veri sayisi : {train["Lise Adi"].isna().sum()}")
train["Lise Adi"].value_counts()

Lise Adi none veri sayisi : 659


Lise Adi
Atatürk Lisesi                                               199
Açık öğretim lisesi                                          163
Açık Öğretim Lisesi                                          157
Cumhuriyet Anadolu Lisesi                                    147
Atatürk Anadolu Lisesi                                       143
                                                            ... 
özel İstanbul Beylikdüzü temel lisesi                          1
GemlikTİcaret Meslek Lisesi                                    1
Beşir Balcıoğlu Anadolu Lisesi-KRK Szilady Aron Gimnazium      1
Istanbulluoğlu Anadolu Öğretmen lisesi                         1
Samsun Terme Karadeniz mesleki teknik Anadolu lisesi           1
Name: count, Length: 33881, dtype: int64

In [732]:
train.loc[(train["Lise Adi"] == '------') | (train["Lise Adi"].isna()), "Lise Adi"] = train["Lise Adi Diger"]

In [733]:
print(f"Lise Adi none veri sayisi : {train["Lise Adi"].isna().sum()}")
train["Lise Adi"].value_counts()

Lise Adi none veri sayisi : 659


Lise Adi
Atatürk Lisesi                                               199
Açık öğretim lisesi                                          163
Açık Öğretim Lisesi                                          157
Cumhuriyet Anadolu Lisesi                                    147
Atatürk Anadolu Lisesi                                       143
                                                            ... 
özel İstanbul Beylikdüzü temel lisesi                          1
GemlikTİcaret Meslek Lisesi                                    1
Beşir Balcıoğlu Anadolu Lisesi-KRK Szilady Aron Gimnazium      1
Istanbulluoğlu Anadolu Öğretmen lisesi                         1
Samsun Terme Karadeniz mesleki teknik Anadolu lisesi           1
Name: count, Length: 33881, dtype: int64